# Data Download & Processing

The objective of this script is to create a common dataset for our analysis. It includes the following functions:
- download MTA bus data from May 1 to 31, 2017
- process data in each file 
- select only M15 and M15 SBS buses
- concatenate daily dataset into one consolidated dataset

---

# Download MTA Bus Data

- Download
- Unzip
- Remove zipped files

In [1]:
# Download Bus Time for all May, 2017
# Only do it at set up


for i in range(1, 32):
    
    if i < 10: 
        file = 'bus_time_2017050' + str(i) + '.csv.xz'
    else: 
        file = 'bus_time_201705' + str(i) + '.csv.xz'
    
    url = 'http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2017/2017-05/' + file 
    
    print("--- Downloading MTA Bus Data of May %i, 2017 ---" %i)
    !wget -nc $url -P './download'
    
    print("--- Unzipping MTA Bus Data of May %i, 2017 ---" %i)
    path = "./download/" + file
    !7za e $path -o./data -aoa


--- Downloading MTA Bus Data of May 1, 2017 ---
File “./download/bus_time_20170501.csv.xz” already there; not retrieving.

--- Unzipping MTA Bus Data of May 1, 2017 ---

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,64 CPUs Intel(R) Xeon(R) CPU E5-4640 0 @ 2.40GHz (206D7),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./download                    1 file, 45612524 bytes (44 MiB)

Extracting archive: ./download/bus_time_20170501.csv.xz
--
Path = ./download/bus_time_20170501.csv.xz
Type = xz
Physical Size = 45612524
Method = LZMA2:26 CRC64
Streams = 1
Blocks = 1

      0% - bus_time_20170501.c                            1% - bus_time_20170501.c                            3% - bus_time_20170501.c                            4% - bus_time_20170501.c                            7% - bus_time_20170501.c                            9% - bus_time_20170501.c                           12% - bus_

      2% - bus_time_20170504.c                            5% - bus_time_20170504.c                            9% - bus_time_20170504.c                           12% - bus_time_20170504.c                           15% - bus_time_20170504.c                           19% - bus_time_20170504.c                           22% - bus_time_20170504.c                           26% - bus_time_20170504.c                           29% - bus_time_20170504.c                           33% - bus_time_20170504.c                           36% - bus_time_20170504.c                           40% - bus_time_20170504.c                           43% - bus_time_20170504.c                           47% - bus_time_20170504.c                           50% - bus_time_20170504.c                           53% - bus_time_20170504.c                           57% - bus_time_20170504.c                           60% - bus_time_20170504.c                           63% - bus_time_20170504.c                           67% - b

--
Path = ./download/bus_time_20170508.csv.xz
Type = xz
Physical Size = 43411684
Method = LZMA2:26 CRC64
Streams = 1
Blocks = 1

      1% - bus_time_20170508.c                            2% - bus_time_20170508.c                            4% - bus_time_20170508.c                            6% - bus_time_20170508.c                            9% - bus_time_20170508.c                           12% - bus_time_20170508.c                           15% - bus_time_20170508.c                           19% - bus_time_20170508.c                           21% - bus_time_20170508.c                           24% - bus_time_20170508.c                           28% - bus_time_20170508.c                           31% - bus_time_20170508.c                           35% - bus_time_20170508.c                           38% - bus_time_20170508.c                           39% - bus_time_20170508.c                           42% - bus_time_20170508.c                           45% - bus_time_20170508.c         

      1% - bus_time_20170511.c                            5% - bus_time_20170511.c                            8% - bus_time_20170511.c                           11% - bus_time_20170511.c                           15% - bus_time_20170511.c                           18% - bus_time_20170511.c                           21% - bus_time_20170511.c                           24% - bus_time_20170511.c                           26% - bus_time_20170511.c                           29% - bus_time_20170511.c                           33% - bus_time_20170511.c                           36% - bus_time_20170511.c                           40% - bus_time_20170511.c                           44% - bus_time_20170511.c                           47% - bus_time_20170511.c                           51% - bus_time_20170511.c                           54% - bus_time_20170511.c                           58% - bus_time_20170511.c                           61% - bus_time_20170511.c                           65% - b

      1% - bus_time_20170515.c                            4% - bus_time_20170515.c                            6% - bus_time_20170515.c                            9% - bus_time_20170515.c                           12% - bus_time_20170515.c                           16% - bus_time_20170515.c                           19% - bus_time_20170515.c                           22% - bus_time_20170515.c                           26% - bus_time_20170515.c                           29% - bus_time_20170515.c                           33% - bus_time_20170515.c                           36% - bus_time_20170515.c                           39% - bus_time_20170515.c                           43% - bus_time_20170515.c                           47% - bus_time_20170515.c                           50% - bus_time_20170515.c                           53% - bus_time_20170515.c                           57% - bus_time_20170515.c                           60% - bus_time_20170515.c                           64% - b

      0% - bus_time_20170518.c                            1% - bus_time_20170518.c                            4% - bus_time_20170518.c                            5% - bus_time_20170518.c                            8% - bus_time_20170518.c                           12% - bus_time_20170518.c                           15% - bus_time_20170518.c                           18% - bus_time_20170518.c                           21% - bus_time_20170518.c                           25% - bus_time_20170518.c                           28% - bus_time_20170518.c                           31% - bus_time_20170518.c                           35% - bus_time_20170518.c                           38% - bus_time_20170518.c                           42% - bus_time_20170518.c                           45% - bus_time_20170518.c                           48% - bus_time_20170518.c                           51% - bus_time_20170518.c                           55% - bus_time_20170518.c                           59% - b

--
Path = ./download/bus_time_20170522.csv.xz
Type = xz
Physical Size = 43026068
Method = LZMA2:26 CRC64
Streams = 1
Blocks = 1

      2% - bus_time_20170522.c                            3% - bus_time_20170522.c                            6% - bus_time_20170522.c                           10% - bus_time_20170522.c                           13% - bus_time_20170522.c                           17% - bus_time_20170522.c                           20% - bus_time_20170522.c                           24% - bus_time_20170522.c                           27% - bus_time_20170522.c                           31% - bus_time_20170522.c                           35% - bus_time_20170522.c                           38% - bus_time_20170522.c                           42% - bus_time_20170522.c                           45% - bus_time_20170522.c                           49% - bus_time_20170522.c                           52% - bus_time_20170522.c                           56% - bus_time_20170522.c         

      0% - bus_time_20170525.c                            1% - bus_time_20170525.c                            2% - bus_time_20170525.c                            3% - bus_time_20170525.c                            6% - bus_time_20170525.c                            9% - bus_time_20170525.c                           12% - bus_time_20170525.c                           15% - bus_time_20170525.c                           18% - bus_time_20170525.c                           22% - bus_time_20170525.c                           25% - bus_time_20170525.c                           28% - bus_time_20170525.c                           32% - bus_time_20170525.c                           35% - bus_time_20170525.c                           39% - bus_time_20170525.c                           42% - bus_time_20170525.c                           46% - bus_time_20170525.c                           49% - bus_time_20170525.c                           52% - bus_time_20170525.c                           55% - b

--
Path = ./download/bus_time_20170529.csv.xz
Type = xz
Physical Size = 27052292
Method = LZMA2:26 CRC64
Streams = 1
Blocks = 1

      2% - bus_time_20170529.c                            3% - bus_time_20170529.c                            9% - bus_time_20170529.c                           11% - bus_time_20170529.c                           16% - bus_time_20170529.c                           22% - bus_time_20170529.c                           27% - bus_time_20170529.c                           32% - bus_time_20170529.c                           38% - bus_time_20170529.c                           43% - bus_time_20170529.c                           47% - bus_time_20170529.c                           53% - bus_time_20170529.c                           58% - bus_time_20170529.c                           63% - bus_time_20170529.c                           68% - bus_time_20170529.c                           74% - bus_time_20170529.c                           79% - bus_time_20170529.c         

In [2]:
file = 'http://web.mta.info/developers/data/nyct/bus/google_transit_manhattan.zip'
!wget -nc $file -P './download'
!cd download; unzip -o google_transit_manhattan.zip

--2017-11-28 13:51:58--  http://web.mta.info/developers/data/nyct/bus/google_transit_manhattan.zip
Resolving web.mta.info... 23.215.130.154, 23.215.130.192
Connecting to web.mta.info|23.215.130.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: “./download/google_transit_manhattan.zip”

    [                      <=>              ] 13,385,446  3.07M/s   in 4.5s    

2017-11-28 13:52:03 (2.86 MB/s) - “./download/google_transit_manhattan.zip” saved [13385446]

Archive:  google_transit_manhattan.zip
  inflating: agency.txt              
  inflating: calendar.txt            
  inflating: calendar_dates.txt      
  inflating: stops.txt               
  inflating: trips.txt               
  inflating: stop_times.txt          
  inflating: shapes.txt              
  inflating: routes.txt              


In [60]:
'''

path = './download/gtfs_nyct_bus_20150103.zip'
!7za e $path -o./data

'''

"\n\npath = './download/gtfs_nyct_bus_20150103.zip'\n!7za e $path -o./data\n\n"

---

In [3]:
import pandas as pd
import numpy as np

In [4]:
trips = pd.read_csv("./download/trips.txt")
stops = pd.read_csv("./download/stops.txt")
schedules = pd.read_csv("./download/stop_times.txt")

### Process and Concatenate 3 Days of Bus Data for Testing

In [5]:
import os
os.path.isfile

<function genericpath.isfile>

In [6]:
# Basic Feature Engineering
# Outputs:
# 1) 31 daily dataset with all bus lines
# 2) 31 daily M15 datasets
# 3) 31 daily M15SBS datasets

import pandas as pd
for i in range(1, 32):
    try:
        if i < 10: 
            file_base = './data/bus_time_2017050' + str(i)
            file = file_base + ".csv"
        else: 
            file_base = './data/bus_time_201705' + str(i)
            file = file_base + ".csv"
            
        if os.path.isfile(file_base + "_m15_filtered.csv"):
            print("Output already exists at {}".format(file_base + "_m15_filtered.csv"))
            print("--- Moving on --- ")
            continue
            
        # ingest into a data frame
        print(" --- Processing %s ---" %file)
        df = pd.read_csv(file)
        df = df[df['block_assigned'] == 1]
        # convert timestamp to DateTime
        df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format = True)
        df['Day'] = df['timestamp'].apply(lambda x: x.day)
        df['Hour'] =df['timestamp'].apply(lambda x: x.hour)
        df['DOW'] = df['timestamp'].dt.dayofweek

        days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
        df['DOW_Label'] = df['DOW'].apply(lambda x: days[x])
        
        print(" --- Done (1/3) ---")
        
        #throw away trips with <15 reports
        df = df.groupby(df.trip_id).filter(lambda group: len(group) > 15)
        
        print(" --- Done (2/3) --- ")
        m15 = df[df.trip_id.str.contains("_M15")]
        m15_sbs = df[df.trip_id.str.contains("_SBS15")]
        
        m15.to_csv(file_base + "_m15_filtered.csv")
        m15_sbs.to_csv(file_base + "_m15_sbs_filtered.csv")
        print(" --- Done (3/3) --- ")
        del df
        del m15
        del m15_sbs
        print("--- Done (cleanup) --- ")
        
    except Exception as ex:
        print(ex)
        pass

 --- Processing ./data/bus_time_20170501.csv ---


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
--- Done (cleanup) --- 
 --- Processing ./data/bus_time_20170502.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
--- Done (cleanup) --- 
 --- Processing ./data/bus_time_20170503.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
--- Done (cleanup) --- 
 --- Processing ./data/bus_time_20170504.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
--- Done (cleanup) --- 
 --- Processing ./data/bus_time_20170505.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
--- Done (cleanup) --- 
 --- Processing ./data/bus_time_20170506.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
--- Done (cleanup) --- 
 --- Processing ./data/bus_time_20170507.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
--- Done (cleanup) --- 
 --- Processing ./data/bus_time_20170508.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3

In [12]:
# Concatenate all M15 datasets of May 2017

m15_final_df = pd.DataFrame()
m15_sbs_final_df = pd.DataFrame()

for i in range(1, 32):
    if i < 10: 
        m15_file = './data/bus_time_2017050' + str(i) + "_m15_filtered.csv"
        m15_sbs_file = './data/bus_time_2017050' + str(i) + "_m15_sbs_filtered.csv"
    else: 
        m15_file = './data/bus_time_201705' + str(i) + "_m15_filtered.csv"
        m15_sbs_file = './data/bus_time_201705' + str(i) + "_m15_sbs_filtered.csv"
    
    # read in each daily file
    m15_df = pd.read_csv(m15_file)
    m15sbs_df = pd.read_csv(m15_sbs_file)
    
    # concatenate daily fales
    m15_final_df = pd.concat([m15_final_df, m15_df])
    m15_sbs_final_df = pd.concat([m15_sbs_final_df, m15sbs_df])
    
    print("---  concate {} ---".format(m15_file))
    print("---  concate {} ---".format(m15_sbs_file))
    
    # clean up 
    del m15_df
    del m15sbs_df

---  concate ./data/bus_time_20170501_m15_filtered.csv ---
---  concate ./data/bus_time_20170501_m15_sbs_filtered.csv ---
---  concate ./data/bus_time_20170502_m15_filtered.csv ---
---  concate ./data/bus_time_20170502_m15_sbs_filtered.csv ---
---  concate ./data/bus_time_20170503_m15_filtered.csv ---
---  concate ./data/bus_time_20170503_m15_sbs_filtered.csv ---
---  concate ./data/bus_time_20170504_m15_filtered.csv ---
---  concate ./data/bus_time_20170504_m15_sbs_filtered.csv ---
---  concate ./data/bus_time_20170505_m15_filtered.csv ---
---  concate ./data/bus_time_20170505_m15_sbs_filtered.csv ---
---  concate ./data/bus_time_20170506_m15_filtered.csv ---
---  concate ./data/bus_time_20170506_m15_sbs_filtered.csv ---
---  concate ./data/bus_time_20170507_m15_filtered.csv ---
---  concate ./data/bus_time_20170507_m15_sbs_filtered.csv ---
---  concate ./data/bus_time_20170508_m15_filtered.csv ---
---  concate ./data/bus_time_20170508_m15_sbs_filtered.csv ---
---  concate ./data/bus_

In [16]:
# Dataset Validation
m15_final_df.shape

(645808, 17)

In [20]:
m15_final_df['timestamp'] = pd.to_datetime(m15_final_df['timestamp'], infer_datetime_format= True)
print(m15_final_df['timestamp'].min())
print(m15_final_df['timestamp'].max())

2017-05-01 00:00:00
2017-05-31 23:59:41


In [ ]:
m15_final_df.sample()

In [21]:
m15_sbs_final_df['timestamp'] = pd.to_datetime(m15_sbs_final_df['timestamp'], infer_datetime_format= True)
print(m15_sbs_final_df['timestamp'].min())
print(m15_sbs_final_df['timestamp'].max())

2017-05-01 00:00:01
2017-05-31 23:59:41


In [22]:
# Dataset Export
m15_final_df.to_csv("./data/m15_final.csv")
m15_sbs_final_df.to_csv("./data/m15_sbs_final.csv")